### Notebook to generate DMD approximations of the flow around a cylinder problem

In [ ]:
## Load all modules
%matplotlib inline
import numpy as np
import scipy
from scipy import linalg
import matplotlib.pyplot as plt
import scipy.interpolate as interpolate
import gc
import os,sys
from importlib import reload


# Plot parameters
plt.rc('font', family='serif')
plt.rcParams.update({'font.size': 20,
                     'lines.linewidth': 2,
                     'axes.labelsize': 16, 
                     'axes.titlesize': 20,
                     'xtick.labelsize': 16,
                     'ytick.labelsize': 16,
                     'legend.fontsize': 16,
                     'axes.linewidth': 2})

basedir = os.getcwd()
srcdir = os.path.join(basedir,'../pynirom/dmd/')
workdir = os.path.join(basedir,'../examples/')
datadir = os.path.join(basedir,'../data/')
nirom_data_dir = os.path.join(basedir,'../data/')
figdir = os.path.join(basedir,'../figures/dmd/')


import pynirom
from pynirom.dmd import main as dmd
from pynirom.dmd import plotting as pu


In [ ]:
## ----- Load spatial mesh from saved numpy file -----
## Please download the data files from 
## https://drive.google.com/drive/folders/1QG4dyoil5QGHjx3d1L3t0S6lsTGS7Vh0?usp=sharing
## and place them in <data_dir>

meshfile = 'OF_cylinder_mesh_Nn14605_Ne28624.npz'
mesh_load = np.load(datadir+meshfile)
nodes = mesh_load['nodes']; triangles = mesh_load['elems']
Nn = nodes.shape[0]; Ne = triangles.shape[0]
node_ind = mesh_load['node_ind']; elem_ind = mesh_load['elem_ind']

print("OpenFOAM mesh has %d nodes and %d elements"%(Nn, Ne))
print("Mesh element keys are : " + str(list(mesh_load.keys())))
del mesh_load

x = nodes[:,0]; y = nodes[:,1]

In [ ]:
## Prepare training snapshots
soln_names = ['p', 'v_x', 'v_y']
comp_names={0:'p',1:'v_x',2:'v_y'}
Nc=3 
snap_start = 1250
T_end = 5.0   ### 5 seconds 

data = np.load(datadir+'cylinder_Re100.0_Nn14605_Nt3001.npz')
print("Solution component keys are : " + str(list(data.keys())))

snap_data = {}
for key in soln_names:
    snap_data[key] = data[key][:,snap_start:]

Tsim = data['time']
times_offline = data['time'][snap_start:]
DT = (times_offline[1:] - times_offline[:-1]).mean()
Nt = times_offline.size
print('Loaded {0} snapshots of dimension {1} for h,u and v, spanning times [{2}, {3}]'.format(
                    snap_data[soln_names[0]].shape[1],snap_data[soln_names[0]].shape[0], 
                    times_offline[0], times_offline[-1]))

## number of steps to skip in selecting training snapshots for SVD basis
snap_incr=4  #(== nDT_skip)

## Normalize the time axis. Required for DMD fitting
tscale = DT*snap_incr            ### Scaling for DMD
times_offline = times_offline/tscale   ## Snapshots DT = 1



## Subsample snapshots for building POD basis
snap_end = np.count_nonzero(times_offline[times_offline <= T_end/tscale])
snap_train = {};
for key in soln_names:
    snap_train[key] = snap_data[key][:,0:snap_end+1:snap_incr]

times_train=times_offline[0:snap_end+1:snap_incr]
Nt_b = times_train.size
print('Using {0} training snapshots for time interval [{1},{2}]'.format(times_train.shape[0], 
                                        times_train[0]*tscale, times_train[-1]*tscale))

del data
gc.collect()

In [ ]:
### Set up the snapshot data matrices X and Y describing the DMD flow map : Y = AX
## For multicomponent systems such Navier Stokes equations (p,u,v),
## the snapshot matrix can be created either by vertically stacking
## solution variables one after another, or by interleaving the
## solution variables at every spatial node.
## In each case, the snapshot matrix has dimensions
## (3 * DOF) X (Time Steps)

interleaved_snapshots = True
X0 = np.zeros((Nc*Nn,Nt_b),'d')

for ivar,key in enumerate(soln_names):   
    if interleaved_snapshots:    ### saving in an interleaved fashion
        X0[ivar::Nc,:] = snap_train[key][:,:]
    else:                        ### saving in a sequential fashion
        X0[ivar*Nn:(ivar+1)*Nn,:] = snap_train[key][:,:]

        
X  = X0[:,:-1]
Xp = X0[:,1:]

In [ ]:
## Set the time steps for online prediction

t0 = times_train[0]
trainT0 = np.searchsorted(times_offline, t0)
trainT = np.searchsorted(times_offline, times_train[-1])
trainP = np.searchsorted(times_offline, 6.0/tscale)

finer_steps = True
long_term = True

if finer_steps and not long_term:
    onl_skip = snap_incr-1
    times_online = times_offline[trainT0:trainT+1:onl_skip]
    N_online = trainT+1
elif long_term and not finer_steps:
    onl_skip = snap_incr
    times_online = times_offline[trainT0:trainP:onl_skip]
    N_online = trainP+1
elif long_term and finer_steps:
    onl_skip = 1 
    times_online = times_offline[trainT0:trainP:onl_skip]
    N_online = trainP+1
Nt_online = times_online.size
print('Trying to simulate interval [{0},{1}] days with {2} steps'.format(t0*tscale,
                                                times_online[-1]*tscale, Nt_online))



In [ ]:
## Create a snapshot matrix for the true solution at 
## testing time points, following the chosen 
## stacking convention

X_true = np.zeros((Nc*Nn,Nt_online),'d')
onl_index = np.searchsorted(times_offline, times_online)
for ivar,key in enumerate(soln_names):
    ### saving in an interleaved fashion
    if interleaved_snapshots:    
        X_true[ivar::Nc,:] = snap_data[key][:,onl_index] 
    ### saving in a sequential fashion
    else:                        
        X_true[ivar*Nn:(ivar+1)*Nn,:] = snap_data[key][:,onl_index]



In [ ]:
def var_string(ky):
    md = ky
    return md

In [ ]:
### Compute the DMD modes
## Using a predetermined fixed number of truncation modes for SVD

# r = 30  #HIGHER TRUNCATION LEVEL
r = 8   #LOWER TRUNCATION LEVEL
t0,dt = times_train[0], times_train[1] - times_train[0]


DMD=dmd.DMDBase(rank=r)
Phi,D,X_app,td,pod_Sigma = DMD.fit_basis(X0, dt_fit = dt,
                                                    t0_fit = times_train[0])
Xdmd = np.zeros((Nn*Nc,Nt_online),'d')
for inx,tn in enumerate(times_online):
    Xdmd[:,inx] = DMD.predict(tn)
print("DMD snapshots computed for %d steps between t = [%.3f, %.3f]"%(Nt_online, 
                                    times_online[0]*tscale, times_online[-1]*tscale))



In [ ]:
### Look at the singular value decay
pu.plot_sing_val(pod_Sigma[:r],6,4)

In [ ]:
## Visualize distribution of DMD e-values
pu.plot_DMD_eval(D,r)

print('Total distance between eigenvalues and unit circle: {}'.format(
    np.sum(np.abs(D.real**2 + D.imag**2 - 1)) ) )


In [ ]:
def plot_dmd_soln(X, Xdmd, Nc, Nt_plot, nodes, elems, trainT0, times_online, comp_names, seed =100, flag = True): 
    
    np.random.seed(seed)
    itime = np.random.randint(0,Nt_plot)
    ivar  = np.random.randint(1,Nc)

    if flag:     ### for interleaved snapshots
        tmp      = Xdmd[ivar::Nc,itime]
        tmp_snap = X[ivar::Nc,itime]
    else:
        tmp      = Xdmd[ivar*Nn:(ivar+1)*Nn,itime]
        tmp_snap = X[ivar*Nn:(ivar+1)*Nn,itime]

    ky = comp_names[ivar]
    tn   = times_online[itime]*tscale
    
    fig  = plt.figure(figsize=(18,15));
    ax1   = fig.add_subplot(3, 1, 1)
    surf1 = ax1.tripcolor(nodes[:,0], nodes[:,1],elems, tmp, cmap=plt.cm.jet)
    ax1.set_title('DMD solution: {0} at t={1:1.2f} seconds, {0} range = [{2:5.3g},{3:4.2g}]'.format(ky,tn,
                                                                        tmp.min(),tmp.max()),fontsize=16)
    plt.axis('off')
    plt.colorbar(surf1, orientation='horizontal',shrink=0.6,aspect=40, pad = 0.03)

    ax2   = fig.add_subplot(3, 1, 2)
    surf2 = ax2.tripcolor(nodes[:,0], nodes[:,1],elems, tmp_snap, cmap=plt.cm.jet)
    ax2.set_title('HFM solution: {0} at t={1:1.2f} seconds, {0} range = [{2:5.3g},{3:4.2g}]'.format(ky,tn,
                                                                    tmp_snap.min(),tmp_snap.max()),fontsize=16)
    plt.axis('off')
    plt.colorbar(surf2, orientation='horizontal',shrink=0.6,aspect=40, pad = 0.03)

    err = tmp-tmp_snap
    ax3   = fig.add_subplot(3, 1, 3)
    surf3 = ax3.tripcolor(nodes[:,0], nodes[:,1],elems, err, cmap=plt.cm.Spectral)
    ax3.set_title('DMD error: {0} at t={1:1.2f} seconds, error range = [{2:5.3g},{3:4.2g}]'.format(ky,tn,
                                                                    err.min(),err.max()),fontsize=16)
    plt.axis('off')
    plt.colorbar(surf3,orientation='horizontal',shrink=0.6,aspect=40, pad = 0.03)


In [ ]:
Nt_plot = np.searchsorted(times_online, times_train[-1])
plot_dmd_soln(X_true, Xdmd, Nc, Nt_plot, nodes, triangles, trainT0, times_online, comp_names, seed=150,flag = True)

In [ ]:
def plot_vel_mag(X, Xdmd, Nc, Nt_plot, nodes, elems, trainT0, times_online, flag = True):
    '''
    Plot the magnitude of the velocity for the true solution, 
    the DMD solution and the error
    '''
    import math
    from math import hypot
    
    np.random.seed(1234)
    itime = np.random.randint(0,Nt_plot)


    if flag:   ## snapshots are stored in an interleaved fashion
        tmp      = np.sqrt(Xdmd[0::Nc,itime]**2 + Xdmd[1::Nc,itime]**2)
        tmp_snap = np.sqrt(X[0::Nc,itime]**2 + X[1::Nc,itime]**2)
    else:
        tmp      = Xdmd[ivar*Nn:(ivar+1)*Nn,itime]
        tmp_snap = X[ivar*Nn:(ivar+1)*Nn,itime]



    tn   = times_online[itime]*tscale
    fig  = plt.figure(figsize=(18,15));
    ax   = fig.add_subplot(3, 1, 1)
    surf = ax.tripcolor(nodes[:,0], nodes[:,1],elems, tmp, cmap=plt.cm.jet)
    ax.set_title('DMD solution: $|u|$ at t={0:1.2f} seconds, $|u|$ range = [{1:5.3g},{2:4.2g}]'.format(tn,
                                                                    tmp.min(),tmp.max()),fontsize=16)
    plt.axis('off')
    plt.colorbar(surf, orientation='horizontal',shrink=0.6,aspect=40, pad = 0.03)


    ax   = fig.add_subplot(3, 1, 2)
    surf = ax.tripcolor(nodes[:,0], nodes[:,1],elems, tmp_snap, cmap=plt.cm.jet)
    ax.set_title('HFM solution: $|u|$ at t={0:1.2f} seconds, $|u|$ range = [{1:5.3g},{2:4.2g}]'.format(tn,
                                                                    tmp_snap.min(),tmp_snap.max()),fontsize=16)
    plt.axis('off')
    plt.colorbar(surf, orientation='horizontal',shrink=0.6,aspect=40, pad = 0.03)

    
    err = tmp-tmp_snap
    rel_err = err/tmp_snap
    ax   = fig.add_subplot(3, 1, 3)
    surf = ax.tripcolor(nodes[:,0], nodes[:,1],elems, err, cmap=plt.cm.Spectral)
    ax.set_title('DMD rel error: $|u|$ at t={0:1.2f} seconds, rel. err. range = [{1:5.3g},{2:4.2g}]'.format(tn,
                                                                    err.min(),err.max()),fontsize=16)
    plt.axis('off')
    plt.colorbar(surf,orientation='horizontal',shrink=0.6,aspect=40, pad = 0.03)



In [ ]:
Nt_plot = np.searchsorted(times_online, times_train[-1])
plot_vel_mag(X_true, Xdmd, Nc, Nt_online, nodes, triangles, trainT0, times_online)

In [ ]:
### Visualize spatial RMS/Relative errors

x_inx = times_online*tscale
tr_mark = np.searchsorted(times_online, times_train[-1])

dmd_rms = DMD.compute_error(X_true, Xdmd, soln_names, metric = 'rms')

vstring = {}
for key in soln_names:
    vstring[key] = var_string(key)
    
pu.plot_DMD_err(dmd_rms, x_inx, soln_names, vstring, mark=tr_mark,)

# os.chdir(figdir)
# plt.savefig('OF_dmd_rms_err_pskip%d_oskip%d_r%d.pdf'%(snap_incr,onl_skip,r),bbox_extra_artists=(lg,), bbox_inches='tight')

In [ ]:
## Saving the ROM model
# os.chdir(nirom_data_dir)
# filename='dmd_rom_cylinder'
# DMD.save_to_disk(filename,DMD)
# os.chdir(work_dir)

In [ ]:
## Save the NIROM solutions in numpy format
os.chdir(nirom_data_dir)
np.savez_compressed('cylinder_online_dmd_%d'%r,dmd=Xdmd, true=X_true,time=times_online,tscale=tscale,
                    interleaved=interleaved_snapshots)

os.chdir(work_dir)